In [ ]:
"""
This notebook is used to 1) turn the txt format articles to the json and PubTator formats for the PubTator Central annotator, and 2) turn the txt format articles to separated txt articles for the NIO annotator.

- txt format: "[PMCID].[database].[id].[Title].[Abstract]."
- json format: {"sourcedb":"database","sourceid":"id","text":"Abstract"}
- PubTator format:
    - [id]|t|[title]
    - [id]|a|[abstract]
"""

In [8]:
import json
path = "./Projects/ptc/Code.Python/txt_to_json/ten_recent.txt"  # "ten_recent.txt" can be changed
output_path = "./Projects/ptc/Code.Python/input/"

In [10]:
with open(path,encoding="utf-8-sig") as f:
    contents = f.read().replace("\t", "").splitlines()

sepa_content = list()
for i in range(len(contents)):
    if i%3 == 0:
        new = list()
        new.append(contents[i])
    elif i%3 == 1:
        new.append(contents[i])
    elif i%3 == 2:
        sepa_content.append(new)
        

In [19]:
## save as json
artitle_num = len(sepa_content)
articles = list()
article_names = list()

for i in range(artitle_num):
    art_dict = dict()
    article = sepa_content[i]
    src_title,abstract = article[0].split("."),article[1]
    
    if src_title[0].startswith("PMC"):   # have full text annotations in PTC
        source_db = src_title[1]
        source_id = src_title[2]
        text = src_title[3]+". "+abstract
    
    elif src_title[0] == "NLM":
        source_db = src_title[0]
        source_id = src_title[1]
        text = src_title[2]+". "+abstract
    
    elif src_title[0].startswith("CCC"): 
        db_id = src_title[0].split(":")
        source_db = db_id[0]
        source_id = db_id[1]
        text = src_title[1]+". "+abstract
    
    else:
        print("Other database:", src_title)
        continue

    art_dict["sourcedb"] = source_db
    art_dict["sourceid"] = source_id
    art_dict["text"] = text  
    
    article_names.append(source_db+"_"+source_id)
    articles.append(art_dict)

with open(output_path+"ten_recent.json", 'w') as json_file:
    json.dump(articles, json_file, indent = 4)
    

In [23]:
## save as PubTator
with open("./Projects/ptc/Code.Python/input/ex.PubTator") as f:
    contents = f.read()

In [26]:
artitle_num = len(sepa_content)
txts = ""

for i in range(artitle_num):
    article = sepa_content[i]
    src_title,abstract = article[0].split("."),article[1]

    if src_title[0].startswith("PMC"):   # have full text annotations in PTC
        source_id = src_title[2]
        title = src_title[3]
        
    elif src_title[0] == "NLM":
        source_id = src_title[1]
        title = src_title[2] 
    
    elif src_title[0].startswith("CCC"): 
        db_id = src_title[0].split(":")
        source_id = db_id[1]
        title = src_title[1]
    
    else:
        print("Other database:", src_title)
        break
    
    txts += source_id+"|t|"+title+"\n"+source_id+"|a|"+abstract+"\n"+"\n"
    
f = open(output_path+"ten_recent.PubTator", "w")
f.write(txts)
f.close()

In [27]:
## for NIO, turn the txt format articles to separated txt articles
nio_path = "./Projects/inputs/articles/04_ptc_ten_recent/articles-txt-format/"

In [44]:
for i in range(len(sepa_content)):
    file_name = article_names[i]
    with open(nio_path+file_name+".txt", 'w') as f:
        content = sepa_content[i]
        title = content[0].split(".")[-2]
        abstract = content[1]
        f.write(title+"."+abstract)